In [ ]:
# 1. Complete an exploratory analysis with visualizations of the data to come up with some interesting thoughts/patterns and describe your findings about the data.
# 2. Develop a Machine Learning model to classify the activity based on features present in theneckband. What would be the most optimal model to run on the Neckband?
# 3. Share your code through GitHub or any other preferred way on or before the interview date.
# 4. Please put a summary of your exploration, model development and resulting into a brief presentation to share back with the interviewers.

In [ ]:
# Setup and Config.

In [42]:
!pip install -r requirements.txt

  Using cached auto-sklearn-0.12.7.tar.gz (6.3 MB)
  Using cached ConfigSpace-0.4.19.tar.gz (106 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached distributed-2021.3.0-py3-none-any.whl (675 kB)
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
  Using cached dask-2021.3.0-py3-none-any.whl (925 kB)
  Using cached msgpack-1.0.2-cp36-cp36m-macosx_10_14_x86_64.whl (73 kB)
  Using cached pynisher-0.6.4.tar.gz (11 kB)
  Using cached pyrfr-0.8.2.tar.gz (296 kB)
  Using cached scikit_learn-0.24.2-cp36-cp36m-macosx_10_13_x86_64.whl (7.2 MB)
  Using cached smac-0.13.1.tar.gz (258 kB)
  Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
  Using cached DateTime-4.3-py2.py3-none-any.whl (60 kB)
  Using cached contextvars-2.4.tar.gz (9.6 kB)
  Using cached immutables-0.16-cp36-cp36m-macosx_10_9_x86_64.whl (56 kB)
  Using cached lazy_import-0.2.2.tar.gz (15 kB)
    ERROR: Command errored out with

In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import datetime
pd.set_option('expand_frame_repr', False)

In [32]:
df = pd.read_csv("../data/IMU.csv", dtype={'activity' : 'category'}, parse_dates=['UnixTime','gps_unixTime'], date_parser=lambda epoch: pd.to_datetime(float(epoch)/1000))

#print(df.head(5))
print("\n\n\nDataframe Info:\n")
print(df.info)
print("\n\n\n DF DTypes:\n")
print(df.dtypes)




Dataframe Info:

<bound method DataFrame.info of                          UnixTime  cumulativeAudio  cumulativeShock   lat_sMins   lon_wMins               gps_unixTime  heading_hundredths  velocity_cm_s  hdop  satellitesUsed  ...  gyro_dps.x  gyro_dps.y  gyro_dps.z  mag_nT.x  mag_nT.y  mag_nT.z  IMUspeed  IMUfSpeed  pitch  activity
0      1970-01-01 00:00:01.532740               19                 5 -27.788683  152.686050 1970-01-01 00:00:01.532740              184.14              0   0.8              10  ...          -1           0          -1    -11686    -11850    -24150        22         10     13         0
1      1970-01-01 00:00:01.532740               19                 5 -27.788683  152.686050 1970-01-01 00:00:01.532740              184.14              0   0.8              10  ...          -1           0          -1    -11686    -11850    -24150        22         10     13         0
2      1970-01-01 00:00:01.532740               19                 5 -27.788683  152.686050 1

In [36]:
# Pandas Profiling FTW!
from pandas_profiling import ProfileReport
profiling_report = ProfileReport(df.sample(n=250000))
profiling_report.to_file(output_file='../analysis_outputs/profiling_report_output_sampled250000.html')
profiling_report = ProfileReport(df.sample(n=100000), minimal=True)
profiling_report.to_file(output_file='../analysis_outputs/profiling_report_output_minimalTrue.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 339.43it/s]


Machine Learning Model


In [41]:
# print autosklearn version
import autosklearn
print('autosklearn: %s' % autosklearn.__version__)

# example of auto-sklearn for a classification dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autosklearn.classification import AutoSklearnClassifier
# define dataset
X, y = df.loc[:, df.columns != 'activity'], df['activity']
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# define search
model = AutoSklearnClassifier(time_left_for_this_task=2*60, per_run_time_limit=30, n_jobs=8)
# perform the search
model.fit(X_train, y_train)
# summarize
print(model.sprint_statistics())
# evaluate best model
y_hat = model.predict(X_test)
acc = accuracy_score(y_test, y_hat)
print("Accuracy: %.3f" % acc)

ModuleNotFoundError: No module named 'autosklearn'